<a href="https://colab.research.google.com/github/davidlealo/sic_ai_2025_jun/blob/main/04pln/clase_28.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clase 28

### Unidad 3 – Capítulo 7 · Samsung Innovation Campus 2025

---

## Objetivo

Esta guía paso a paso introduce los conceptos y técnicas esenciales que viste en la Unidad 3 («Modelo de representación», «Análisis de clasificación» y «Modelado de temas»). Incluye explicaciones detalladas, ejemplos en **Python** y ejercicios prácticos. Está pensada para personas que ya programan pero son nuevas en PLN.

---

## Prerrequisitos rápidos

| Herramienta      | Comando de instalación     |
| ---------------- | -------------------------- |
| **NLTK**         | `pip install nltk`         |
| **scikit‑learn** | `pip install scikit-learn` |
| **gensim**       | `pip install gensim`       |

> **Paso 0.** Crea un entorno virtual (opcional pero recomendado) y activa tu entorno antes de instalar los paquetes.

---

## Tabla de contenidos

1. [Flujo de un proyecto de PLN](#flujo)
2. [Pre‑procesamiento del texto](#prepro)
3. [Modelos de representación](#repr)
4. [Clasificación supervisada](#clasif)
5. [Modelado de temas](#temas)
6. [Proyecto integrador & ejercicios](#proyecto)
7. [Recursos adicionales](#recursos)

---



## 1. Flujo de un proyecto de PLN

1. **Definir el problema** → ¿Clasificar sentimiento? ¿Detectar spam? ¿Extraer temas?
2. **Recolección de datos** → APIs, scraping, Kaggle, archivos internos.
3. **Pre‑procesamiento** → Limpieza, tokenización, lematización, stopwords.
4. **Representación** → BoW, TF‑IDF o embeddings.
5. **Modelado** → Clasificadores supervisados o modelos no supervisados (LDA, LSA).
6. **Evaluación** → Métricas (accuracy, precisión, F1, coherencia de temas…).
7. **Despliegue** → API REST, batch, dashboard, etc.

Cada paso se explica con mayor detalle a continuación.

---



## 2. Pre‑procesamiento del texto

### ¿Por qué limpiar el texto?

Los algoritmos trabajan con números. El pre‑procesamiento normaliza la entrada y reduce ruido, mejorando la calidad de la representación.

### Paso a paso 🛠️

1. **Convertir a minúsculas** – Homogeneiza el texto.
2. **Eliminar URLs** – No aportan significado en la mayoría de tareas.
3. **Quitar puntuación** – Facilita la tokenización.
4. **Tokenizar** – Dividir el texto en palabras o sub‑palabras.
5. **Eliminar stopwords** – Palabras muy frecuentes sin carga semántica («the», «y», «de»…).
6. **Lematizar** – Reducir palabras a su forma canónica ( *running* → *run*).

```python
import nltk, re, string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download("punkt"); nltk.download("wordnet"); nltk.download("stopwords")

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def clean_text(text):
    # 1‑2. normalizar y quitar URLs
    text = re.sub(r"https?://\S+", "", text.lower())

    # 3. quitar puntuación
    text = text.translate(str.maketrans("", "", string.punctuation))

    # 4. tokenizar
    tokens = nltk.word_tokenize(text)

    # 5‑6. stopwords + lematizar
    tokens = [lemmatizer.lemmatize(t) for t in tokens if t not in stop_words]
    return " ".join(tokens)
```

> **Ejercicio 2.1 – Explora el impacto**
>
> 1. Copia 5–10 tweets (o reseñas IMDB).
> 2. Aplica `clean_text` y compara texto original vs. limpiado.
> 3. ¿Qué tokens se eliminaron? ¿Cuáles se transformaron?

---



## 3. Modelos de representación

### 3.0 Exploración de frecuencia de palabras

El **conteo de frecuencias** es la forma más básica de cuantificar texto y suele ser el *primer paso* del análisis exploratorio de datos (EDA) en PLN.

#### ¿Por qué contar palabras?

- **Panorama rápido del corpus** → Identifica temas dominantes y vocabulario característico.
- **Ley de Zipf** → La distribución sigue una cola larga: pocas palabras muy frecuentes y muchas raras. Conocerla ayuda a decidir filtros (`min_df`, `max_df`).
- **Detección de *****stopwords***** y ruido** → Palabras ultra‑frecuentes pueden eliminarse para mejorar modelos.
- **Selección de características** → Elegir el *top‑k* de términos o fijar umbrales de frecuencia reduce dimensionalidad.
- **Validación de limpieza** → Números, URLs o artefactos extraños destacan en los rankings.

#### Paso a paso 🛠️

```python
from collections import Counter
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize

# 1. Corpus ya limpiado con clean_text()
texts = [clean_text(doc) for doc in data.data]

# 2. Tokenizar y aplanar
all_tokens = []
for t in texts:
    all_tokens.extend(word_tokenize(t))

# 3. Contar
freq = Counter(all_tokens)
print(freq.most_common(10))  # Top‑10 palabras

# 4. Visualizar
terms, counts = zip(*freq.most_common(20))
plt.figure(figsize=(10,4))
plt.bar(terms, counts)
plt.xticks(rotation=45, ha='right')
plt.title('Top‑20 palabras más frecuentes')
plt.tight_layout()
plt.show()
```

> **Ejercicio 3.0 – Word cloud**
>
> 1. Instala `pip install wordcloud`.
> 2. Genera una nube de palabras con `WordCloud().generate_from_frequencies(freq)`.
> 3. Cambia el resultado filtrando stopwords distintas y observa el efecto.

---

### 3.1 Bolsa de Palabras (BoW)

**Idea:** Cada documento se convierte en un vector donde la dimensión *j* es la frecuencia de la palabra *j*.

```python
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    "Cats eat fish",
    "Dogs eat meat",
    "Cats and dogs are pets"
]
vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out())      # columnas
print(X_bow.toarray())                         # matriz doc×vocab
```

| Paso | Acción                                                  |
| ---- | ------------------------------------------------------- |
| 1    | Construir vocabulario → `fit`                           |
| 2    | Contar apariciones    → `transform`                     |
| 3    | Obtener matriz esparsa de tamaño *(n\_docs × n\_terms)* |

### 3.2 TF‑IDF

**Fórmula:**\
\(\text{tf‑idf}(d, t) = tf(d,t) \times \log \frac{N}{df(t)}\)

- Penaliza términos muy frecuentes globalmente.

```python
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
X_tfidf = vect.fit_transform(corpus)
print(X_tfidf.toarray())
```

> **Ejercicio 3.1 – N‑gramas y filtros**
>
> - Ajusta `ngram_range=(1,2)` (unigramas + bigramas).
> - Ignora términos raros con `min_df=2`.
> - Observa cómo cambia la matriz.

### 3.3 Word Embeddings (vistazo rápido)

Representaciones densas que capturan similitud semántica.

```python
from gensim.models import Word2Vec
sentences = [doc.split() for doc in corpus]
model = Word2Vec(sentences, vector_size=50, window=5, min_count=1, workers=4)
print(model.wv["cats"][:10])  # 10 primeras dimensiones
```

> Para ir más lejos: `sentence-transformers`, `spaCy`.

---



## 4. Clasificación supervisada

### Caso de uso: Clasificar artículos de noticias «espacio» vs. «medicina»

**Paso 1.** Cargar datos → `fetch_20newsgroups`.

**Paso 2.** Dividir en entrenamiento / prueba.

**Paso 3.** Crear un **pipeline**:

1. Vectorizador (`CountVectorizer` o `TfidfVectorizer`).
2. Clasificador (Naive Bayes, SVM, etc.).

```python
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB

cats = ["sci.med", "sci.space"]
data = fetch_20newsgroups(subset="all", categories=cats, remove=("headers","footers","quotes"))
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

pipe = make_pipeline(TfidfVectorizer(), MultinomialNB())
pipe.fit(X_train, y_train)
print(classification_report(y_test, pipe.predict(X_test), target_names=cats))
```

| Métrica       | Significado                              |
| ------------- | ---------------------------------------- |
| **Accuracy**  | Proporción total de aciertos.            |
| **Precision** | Exactitud de las predicciones positivas. |
| **Recall**    | Cobertura de los positivos reales.       |
| **F1**        | Media armónica de Precision y Recall.    |

> **Ejercicio 4.1 – Comparar modelos**
>
> 1. Sustituye `MultinomialNB` por `LinearSVC()`.
> 2. ¿Quién gana en F1? ¿Por qué podría ocurrir?

---



## 5. Modelado de temas

### 5.1 LSA (Latent Semantic Analysis) con SVD truncado

**Paso a paso**

1. Vectorizar con **TF‑IDF**.
2. Aplicar `TruncatedSVD(n_components=k)` para descomponer la matriz.
3. Cada componente ≈ un **tema** (vector de términos con pesos altos).

```python
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, stop_words="english")
X = vectorizer.fit_transform(data.data)
svd = TruncatedSVD(n_components=5, random_state=42)
lsa = make_pipeline(svd, Normalizer(copy=False))
X_lsa = lsa.fit_transform(X)

terms = vectorizer.get_feature_names_out()
for idx, comp in enumerate(svd.components_):
    terms_in_comp = [terms[i] for i in comp.argsort()[-8:]]
    print(f"Tema {idx}:", terms_in_comp)
```

### 5.2 LDA (Latent Dirichlet Allocation) con gensim

1. **Tokenizar & limpiar**.
2. **Diccionario** `id2word`.
3. **Corpus** (lista de pares `(id, freq)`).
4. Entrenar `LdaModel` especificando `num_topics`.

```python
import gensim.corpora as corpora
from gensim.models import LdaModel
from nltk.tokenize import word_tokenize

docs = [word_tokenize(clean_text(doc)) for doc in data.data]
id2word = corpora.Dictionary(docs)
corpus_g = [id2word.doc2bow(d) for d in docs]

lda = LdaModel(corpus=corpus_g, id2word=id2word, num_topics=5, passes=10, random_state=42)
lda.print_topics(num_words=6)
```

| Símbolo | Significado en el gráfico de placas                                              |
| ------- | -------------------------------------------------------------------------------- |
| **α**   | Parámetro Dirichlet para las distribuciones de temas por documento (\(\theta\)). |
| **β**   | Parámetro Dirichlet para las distribuciones de palabras por tema (\(\phi\)).     |
| **z**   | Tema asignado a una palabra concreta.                                            |
| **w**   | Palabra observada.                                                               |

> **Ejercicio 5.1 – Coherencia de temas** Usa `gensim.models.CoherenceModel` para medir la coherencia y elige el `num_topics` óptimo.

---



## 6. Proyecto integrador & ejercicios finales

1. **Dataset** – Elige uno de Kaggle (*IMDB Reviews*, *News Category*…).
2. **Objetivo** – Sentiment Analysis **o** Topic Modeling.
3. **Guía mínima**:
   1. Limpieza y pre‑procesamiento detallado.
   2. Dos representaciones (BoW vs. TF‑IDF) y compara resultados.
   3. Clasificador (NB / SVM) **o** LDA con curva de coherencia.
   4. Informe (`markdown` o `notebook`) con métricas, gráficos y reflexión.
4. **Bonus** – Implementa un pequeño *streamlit* para probar tu modelo.

---



## 7. Recursos adicionales

- **NLTK Book** – [https://www.nltk.org/book/](https://www.nltk.org/book/) (Cap. 1‑6 → tokenización, clasificación)
- **Machine Learning with Text (J. Zumstein)** – capítulos gratuitos online.
- **Gensim Tutorials** – [https://radimrehurek.com/gensim/auto\_examples/index.html](https://radimrehurek.com/gensim/auto_examples/index.html)
- **Coursera NLP Specialization (Deeplearning.ai)** – curso 1 (vector spaces & BoW).

---

### ¡Ahora sí, manos al teclado!

Esta versión incluye explicaciones paso a paso para cada técnica. Ajusta parámetros, prueba datasets nuevos y profundiza.



In [ ]:
# Revisar http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [ ]:
# Revisar https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html

In [ ]:
# Revisar https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

### Ejercicio

Entrena un modelo de reconocimiento de correo spam

https://www.kaggle.com/datasets/venky73/spam-mails-dataset/data



# Quedamos slide 152